# Exercise 04 : A/B-testing
## Required data

In [1]:
%ls ../data/checking-logs.sqlite

../data/checking-logs.sqlite


## Imports

In [2]:
import pandas as pd
import sqlite3

## Connect to DB

In [3]:
db_connection = sqlite3.connect('../data/checking-logs.sqlite')

## `test` table

In [4]:
pd.read_sql(sql='PRAGMA table_info(test);', con=db_connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [5]:
pd.read_sql(sql='SELECT * FROM test LIMIT 10;', con=db_connection, index_col='index')

,uid,labname,first_commit_ts,first_view_ts
index,,,,
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035


## `deadlines` table

In [6]:
pd.read_sql(sql='PRAGMA table_info(deadlines);', con=db_connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [7]:
pd.read_sql(sql='SELECT * FROM deadlines LIMIT 10;', con=db_connection, index_col='index')

,labs,deadlines
index,,
0,laba04,1587945599
1,laba04s,1587945599
2,laba05,1588550399
4,laba06,1590364799
5,laba06s,1590364799
3,project1,1589673599


## `test_results` table

In [8]:
sql_query="""

SELECT
    'before' AS time,
    AVG(CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    test
INNER JOIN 
    deadlines
ON
    test.labname != 'project1'
    AND test.labname = deadlines.labs
WHERE
    julianday(test.first_commit_ts) < julianday(test.first_view_ts)


UNION


SELECT
    'after' AS time,
    AVG(CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    test
INNER JOIN 
    deadlines
ON
    test.labname != 'project1'
    AND test.labname = deadlines.labs
WHERE
    julianday(test.first_commit_ts) > julianday(test.first_view_ts)
"""

test_results = pd.read_sql(sql=sql_query, con=db_connection)
test_results

,time,avg_diff
0,after,-103.40625
1,before,-60.56250


## `control_results` table

In [9]:
sql_query="""

SELECT
    'before' AS time,
    AVG(CAST((julianday(control.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    control
INNER JOIN 
    deadlines
ON
    control.labname != 'project1'
    AND control.labname = deadlines.labs
WHERE
    julianday(control.first_commit_ts) < julianday(control.first_view_ts)


UNION


SELECT
    'after' AS time,
    AVG(CAST((julianday(control.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    control
INNER JOIN 
    deadlines
ON
    control.labname != 'project1'
    AND control.labname = deadlines.labs
WHERE
    julianday(control.first_commit_ts) > julianday(control.first_view_ts)
"""

control_results = pd.read_sql(sql=sql_query, con=db_connection)
control_results

,time,avg_diff
0,after,-112.710526
1,before,-99.464286
